# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import holoviews as hv 
import json
hv.extension('bokeh', 'matplotlib')
# Import API key
from api_keys import geoapify_key


In [3]:
INPUT_CSV_PATH = "output_data/cities.csv"
MIN_ACCEPTABLE_TMPTR_C = 20
MAX_ACCEPTABLE_CLOUDINESS_PCT = 200
MIN_ACCEPTABLE_CLOUDINESS_PCT = 80

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(INPUT_CSV_PATH)

# Display sample data
print(city_data_df.shape)
city_data_df.dropna(inplace=True)
print(city_data_df.shape)
city_data_df.head(3)

(589, 10)
(586, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.066,-130.1015,24.80,75,92,2.02,PN,1737991430
1,1,invercargill,-46.400,168.3500,10.01,88,96,2.12,NZ,1737991431
2,2,port-aux-francais,-49.350,70.2167,4.49,97,100,11.03,TF,1737991432


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# YOUR CODE HERE
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo= True, 
    tiles= 'OSM',
    color= 'Humidity',
    cmap = 'plasma', 
    size = 'Humidity',
    hover_cols=['City', 'Humidity'],
    title="City Map with Humidity Represented by Point Size"
)
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria 
# [Already done in cell 2] Drop any rows with null values
#create masks
min_tmptr_mask_s = city_data_df['Max Temp'] >= MIN_ACCEPTABLE_TMPTR_C
min_cloudiness_mask_s = city_data_df['Cloudiness'] >= MIN_ACCEPTABLE_CLOUDINESS_PCT
max_cloudiness_mask_s = city_data_df['Cloudiness'] <= MAX_ACCEPTABLE_CLOUDINESS_PCT
# Examine Mask Series
print(min_tmptr_mask_s.head().index)
print(min_tmptr_mask_s.head().values)
for mask in (min_tmptr_mask_s, min_cloudiness_mask_s,max_cloudiness_mask_s):
    print(sum(mask))

filtered_weather_df = city_data_df[min_tmptr_mask_s & min_cloudiness_mask_s & max_cloudiness_mask_s]
print(filtered_weather_df.shape)
filtered_weather_df.head(3)


Index([0, 1, 2, 3, 4], dtype='int64')
[ True False False False False]
275
230
586
(90, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.80,75,92,2.02,PN,1737991430
22,22,pauini,-7.7136,-66.9764,23.30,98,100,0.80,BR,1737991455
42,42,hirekerur,14.4667,75.3833,23.03,35,96,2.98,IN,1737991477


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = filtered_weather_df[columns]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(3)

/var/folders/8q/2xw5tr555j9gnlkvyz4fmc840000gn/T/ipykernel_67417/1804961623.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,75,
22,pauini,BR,-7.7136,-66.9764,98,
42,hirekerur,IN,14.4667,75.3833,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
BASE_URL= "https://api.geoapify.com/v2/places"
RADIUS = 100000
my_parameters = {"categories": "accommodation.hotel", "limit": 1, "apiKey": geoapify_key}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    my_parameters["filter"] = f"circle:{longitude},{latitude},{RADIUS}"
    my_parameters["bias"] = f"proximity:{longitude},{latitude}"

    # Make an API request using the params dictionary
    response = requests.get(BASE_URL, params=my_parameters)
    if not response.ok:
        print(f"error: status code {response.status_code} for {row['City']}")
        break
    # Evaluate the API response into python object
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (IndexError, KeyError):
    #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
pauini - nearest hotel: No hotel found
hirekerur - nearest hotel: Surya Deluxe Lodge
hermanus - nearest hotel: Aloe guest house
san antonio de pale - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
kokoda - nearest hotel: No hotel found
soio - nearest hotel: Hotel Maria Teres
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
port mathurin - nearest hotel: Escale Vacances
utrik - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
metlili chaamba - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: Hotel Águila Real
kitwe - nearest hotel: Milford Lodge
betio village - nearest hotel: Seaman Hostel
carnarvon - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
sarangani - nearest hotel: No hotel found
sinabang - nearest hotel: Kita Surf Resort
tofol - nearest hotel: Nautilus Resort
saint-philippe - nearest hotel: Le Baril
diamantina - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,75,No hotel found
22,pauini,BR,-7.7136,-66.9764,98,No hotel found
42,hirekerur,IN,14.4667,75.3833,35,Surya Deluxe Lodge
50,hermanus,ZA,-34.4187,19.2345,79,Aloe guest house
52,san antonio de pale,GQ,-1.4014,5.6325,69,No hotel found
...,...,...,...,...,...,...
576,makoua,CG,0.0069,15.6333,82,No hotel found
577,malabar,AU,-33.9667,151.2500,88,Coogee Bay Boutique Hotel
579,mili,MH,6.0815,171.7350,83,No hotel found
583,bowen,AU,-20.0167,148.2333,87,Whitsunday Sands Resort


In [42]:
hotel_df["Hotel Name"].value_counts()

Hotel Name
No hotel found               24
Hostal Vivar                  1
Le Samana                     1
Hotel Viseu                   1
Ancasa Royale Hotel           1
                             ..
PALM Hotel & Spa              1
Eagles Tala                   1
Hotel Jasmine Jayapura        1
Hacienda de Gracia Resort     1
Suita                         1
Name: count, Length: 67, dtype: int64

In [31]:
my_parameters
#filter=circle:-87.770231,41.878968,5000
response.url

'https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=38b36aab9e5a44b7b3ccbe3ef5a2cd04&filter=circle%3A-130.1015%2C-25.066%2C100000&bias=proximity%3A-130.1015%2C-25.066'

In [21]:
'https://api.geoapify.com/v2/places?categories=accomodation.hotel
&limit=1&apiKey=38b36aab9e5a44b7b3ccbe3ef5a2cd04
&filter=circle%3A-130.1015%2C-25.066%2C100000
&bias=point%3A-130.1015%2C-25.066'
#bias=proximity:41.2257145,52.971411


'https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=38b36aab9e5a44b7b3ccbe3ef5a2cd04&filter=circle%3A-130.1015%2C+-25.066%2C+100000&bias=point%3A-130.1015%2C-25.066'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# Configure the map plot

map_plot_1 = hotel_df.hvplot.points(
      "Lng",
      "Lat",
      geo = True,
      tiles = "OSM",
    hover_cols=['City', 'Humidity'],
      color = ''
      )
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

In [45]:
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo= True, 
    tiles= 'OSM',
    color= 'Humidity',
    cmap = 'plasma', 
    size = 'Humidity',
    hover_cols=['City', 'Humidity'],
    title="City Map with Humidity Represented by Point Size"
)
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)